### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import encode_columns, svm_model, accuracy_calculator, random_forest_model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import ast

### Loading the dataset using Pandas
The data is found in development.csv (development set): a comma-separated values file containing the records from the development set. This portion does have the action and object columns, which you should use to obtain the labels to train and validate your models.
The dataset consists in a collection of audio file in a WAV format. 

Each record is characterized by several attributes. The following is a short description for each of them.
- path: the path of the audio file.
- speakerId: the id of the speaker.
- action: the type of action required through the intent.
- object: the device involved by intent.
- Self-reported fluency level: the speaking fluency of the speaker.
- First Language spoken: the first language spoken by the speaker.
- Current language used for work/school: the main language spoken by the speaker during daily activities.
- gender: the gender of the speaker.
- ageRange: the age range of the speaker.

In [2]:
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")

In [3]:
y = df.values[:,3:5].sum(axis=1)
y

array(['change languagenone', 'activatemusic', 'deactivatelights', ...,
       'deactivatelights', 'deactivatelights', 'increasevolume'],
      dtype=object)

In [4]:
column_names = ["Self-reported fluency level ","First Language spoken", "Current language used for work/school", "speakerId", "gender","ageRange"]


In [5]:
encode_columns(df_eval, column_names)

In [6]:

encode_columns(df,column_names)

In [7]:
df_eval["mfccs"] = df.apply(lambda row: librosa.feature.mfcc(y=librosa.load(row["path"])[0], sr=librosa.load(row["path"])[1]), axis=1)

In [8]:
df["mfccs"] = df.apply(lambda row: librosa.feature.mfcc(y=librosa.load(row["path"])[0], sr=librosa.load(row["path"])[1]), axis=1)

In [ ]:
for i, row in df_eval.iterrows():
    for j, array in enumerate(row['mfccs']):
        mean_array, std_array = np.mean(array), np.std(array)
        df_eval.at[i, f'mfcc_mean{j+1}'] = mean_array

In [ ]:
for i, row in df.iterrows():
    for j, array in enumerate(row['mfccs']):
        mean_array, std_array = np.mean(array), np.std(array)
        df.at[i, f'mfcc_mean{j+1}'] = mean_array

In [ ]:
features = ['Self-reported fluency level ',
       'First Language spoken', 'Current language used for work/school',
       'speakerId', 'gender', 'ageRange', 'mfcc_mean1', 'mfcc_mean2',
       'mfcc_mean3', 'mfcc_mean4', 'mfcc_mean5', 'mfcc_mean6', 'mfcc_mean7',
       'mfcc_mean8', 'mfcc_mean9', 'mfcc_mean10', 'mfcc_mean11', 'mfcc_mean12',
       'mfcc_mean13', 'mfcc_mean14', 'mfcc_mean15', 'mfcc_mean16',
       'mfcc_mean17', 'mfcc_mean18', 'mfcc_mean19', 'mfcc_mean20']

In [ ]:
X_eval = df_eval[features].copy()
X_eval

In [ ]:

X = df[features].copy()
X

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# y_pred_eval = svm_model(X, y, X_eval)

# predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
# predictions.to_csv('my_data.csv', index=True)

In [ ]:
# y_pred = svm_model(X_train, y_train, X_test)
# svm_accuracy = accuracy_calculator(y_test, y_pred)

In [ ]:
# svm_accuracy

In [ ]:
y_pred_eval = random_forest_model(X , X_eval, y)

predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
predictions.to_csv('predictions.csv', index=True)


In [ ]:
# y_pred = random_forest_model(X_train, X_test, y_train)
# random_forest_accuracy = accuracy_calculator(y_test, y_pred)
# random_forest_accuracy

In [ ]:
## Implementation using k-fold
from sklearn.model_selection import KFold, cross_val_score
# define the number of folds and whether to shuffle the data
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# create an instance of the SVM model
clf = SVC()

# use cross_val_score function to perform k-fold cross-validation
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

# print the mean accuracy and standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# shuffle the data before performing k-fold cross validation
X, y = shuffle(X, y)

# perform k-fold cross validation with 5 folds
scores = cross_val_score(clf, X, y, cv=5)

# calculate the mean accuracy of the model across all folds
accuracy = np.mean(scores)
print("Accuracy:", accuracy)
